In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import (LabelEncoder, KBinsDiscretizer, StandardScaler, MinMaxScaler)
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import Adam, Nadam
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.metrics import confusion_matrix
import sklearn
import tensorflow as tf
from google.colab import files
import pickle
from sklearn.linear_model import (LogisticRegression, LogisticRegressionCV)
from sklearn.datasets import make_classification
from sklearn.ensemble import (GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor)
from imblearn.over_sampling import SMOTE
import collections
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import itertools
import random
from random import sample
from pandas import DataFrame
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import warnings
import sys

if not sys.warnoptions:
    warnings.simplefilter("ignore")

warnings.simplefilter(action='ignore', category=FutureWarning)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def get_data(list_of_files):
  X_ls = []
  Y_ls = []
  Y_label = []
  for filename in list_of_files:
    if filename[-1] == 'x':
      df = pd.read_excel(filename)
    else:
      df = pd.read_csv(filename)
    df = df.dropna()

    if len(df.columns) > 2:

      Y = df.iloc[:,-1]
      Y = np.array(Y).reshape(-1, 1)
      scaler = MinMaxScaler()
      Y = pd.DataFrame(scaler.fit_transform(Y))
      Y_ls.append(Y)
      
      # getting Y_label
      yd = DataFrame(Y)
      y = yd.values
      kmeans = KMeans(n_clusters=6).fit(y)
      classified_data = kmeans.labels_
      df_processed = yd.copy()
      df_processed['Cluster Class'] = pd.Series(classified_data, index=df_processed.index)
      y_class = df_processed.iloc[:,-1]
      Y_label.append(y_class)

      # getting X_ls
      X = df.iloc[:, :-1]
      X = X._get_numeric_data()
      X_ls.append(X)

  return(X_ls,Y_ls,Y_label)

In [ ]:
# for cp
def get_data(list_of_files):
  X_ls = []
  Y_ls = []
  
  for filename in list_of_files:
    if filename[-1] == 'x':
      df = pd.read_excel(filename)
    else:
      df = pd.read_csv(filename)
    df = df.dropna()

    # getting Y_ls
    Y = df.iloc[:,-1]
    Y = np.array(Y).reshape(-1, 1)
    scaler = MinMaxScaler()
    Y = pd.DataFrame(scaler.fit_transform(Y))
    Y_ls.append(Y)

    # getting X_ls
    X = df.iloc[:, :-1]
    X = X._get_numeric_data()
    X_ls.append(X)

  return(X_ls,Y_ls)

In [ ]:
import glob
test_files = glob.glob('/content/drive/My Drive/TESTING DATASETS/reg_binary/*.csv')
test_files_xlsx = glob.glob('/content/drive/My Drive/TESTING DATASETS/reg_binary/*.xlsx')
test_files.extend(test_files_xlsx)   # create the list of file

# X_ls, Y_ls, Y_label = get_data(test_files)
X_ls, Y_ls = get_data(test_files)

In [ ]:
# prepare test data

def scale(X, y):
  '''
  scale the test data
  '''
  scaled_X = preprocessing.MinMaxScaler().fit_transform(X)
  X = pd.DataFrame(data = scaled_X, columns = X.columns)

  scaled_data = X.copy()
  scaled_data['class'] = y

  return X, scaled_data

def make_seq(inc):
  x = []
  start = 0
  end = 100

  while start < end:
    start += inc 
    start = round(start,2)
    x.append(start)
  return(x)

def percentiles(x, b):
  # return np.percentile(x, np.linspace(0,100,num = b))
  c = 100/b
  return np.percentile(x, make_seq(c))

def qsa(x, b):
  '''
  return a list of length b within range (lb, ub) representing feature x
  '''
  width = 20 / b
  x = np.sort(x)
  res = [0] * b
  for i in x:
    num = int((i - -10) // width)
    if num == b:
      res[b-1] += 1
    else:
      res[num] += 1
  res = [i / len(x) for i in res]
  return res

def bin_edge(X, scaled_data, lb, ub, b):
  '''
  return a representation of features in a dataset
  '''
  col_names = X.columns

  cls0 = scaled_data[scaled_data['class'] == 0]
  cls1 = scaled_data[scaled_data['class'] == 1]
  cls2 = scaled_data[scaled_data['class'] == 2]
  cls3 = scaled_data[scaled_data['class'] == 3]
  cls4 = scaled_data[scaled_data['class'] == 4]
  cls5 = scaled_data[scaled_data['class'] == 5]
  
  #scale to range(-10,10)
  cls0_scaler = MinMaxScaler(feature_range=(lb, ub)).fit_transform(cls0[col_names])
  cls0_scaler = pd.DataFrame(data = cls0_scaler, columns = col_names)
  cls1_scaler = MinMaxScaler(feature_range=(lb, ub)).fit_transform(cls1[col_names])
  cls1_scaler = pd.DataFrame(data = cls1_scaler, columns = col_names)
  cls2_scaler = MinMaxScaler(feature_range=(lb, ub)).fit_transform(cls2[col_names])
  cls2_scaler = pd.DataFrame(data = cls2_scaler, columns = col_names)
  cls3_scaler = MinMaxScaler(feature_range=(lb, ub)).fit_transform(cls3[col_names])
  cls3_scaler = pd.DataFrame(data = cls3_scaler, columns = col_names)
  cls4_scaler = MinMaxScaler(feature_range=(lb, ub)).fit_transform(cls4[col_names])
  cls4_scaler = pd.DataFrame(data = cls4_scaler, columns = col_names)
  cls5_scaler = MinMaxScaler(feature_range=(lb, ub)).fit_transform(cls5[col_names])
  cls5_scaler = pd.DataFrame(data = cls5_scaler, columns = col_names)

  if len(col_names) > 1:
    combos = list(itertools.combinations(col_names,2))
  else:
    combos = []

  training_data = []
  
  for combo in combos:

    bin_all = []

    bin_all.extend(qsa(cls0_scaler[combo[0]], b))
    bin_all.extend(qsa(cls1_scaler[combo[0]], b))
    bin_all.extend(qsa(cls2_scaler[combo[0]], b))
    bin_all.extend(qsa(cls3_scaler[combo[0]], b))
    bin_all.extend(qsa(cls4_scaler[combo[0]], b))
    bin_all.extend(qsa(cls5_scaler[combo[0]], b))

    bin_all.extend(qsa(cls0_scaler[combo[1]], b))
    bin_all.extend(qsa(cls1_scaler[combo[1]], b))
    bin_all.extend(qsa(cls2_scaler[combo[1]], b))
    bin_all.extend(qsa(cls3_scaler[combo[1]], b))
    bin_all.extend(qsa(cls4_scaler[combo[1]], b))
    bin_all.extend(qsa(cls5_scaler[combo[1]], b))
    
    training_data.append(bin_all)

  return training_data

def create_testdata(X, Y, bins):
  '''
  return test data to be fed into mlps
  '''
  test_data = []
  
  X, scaled_data = scale(X, Y)
  test = bin_edge(X, scaled_data, -10, 10, bins)
  test_data.extend(test)
  
  return test_data

In [ ]:
# prepare test data for cp

def scale(variable):
  """Scales the original variable from 0 to 1"""
  data = np.array(variable).reshape(-1,1)
  scaled_data = preprocessing.MinMaxScaler().fit_transform(data)
  scaled_data = pd.DataFrame(data = scaled_data)
  scaled_data = scaled_data.copy()
  return(scaled_data)

def scale_data(X):
  '''
  scale the test data
  '''
  scaled_X = preprocessing.MinMaxScaler().fit_transform(X)
  X = pd.DataFrame(data = scaled_X, columns = X.columns)

  return X

def ranging(variable, bins):
  quantiles_list = []
  quantiles = make_seq(inc=bins)
  for i in quantiles:
    quantiles_list.append(np.percentile(variable, i))
  return(quantiles_list)

def make_seq(inc):
  x = []
  start = 0
  end = 100

  while start < end:
    start += inc 
    start = round(start,2)
    x.append(start)
  return(x)

def conditional_binning(data,Y,X_q,Y_q):
  FXY = []
  FY = []

  oldy_q = 0
  for y_q in Y_q:
    if oldy_q == 0:
      count = sum(1 for y in Y if y <= y_q and y >= oldy_q )
    else:
      count = sum(1 for y in Y if y <= y_q and y > oldy_q )
    oldy_q = y_q
    FY.append(count)


  oldx_q = 0
  for x_q in X_q:
    oldy_q = 0
    for i,y_q in enumerate(Y_q):
      
      if oldx_q == 0 and oldy_q == 0:
        count = sum(1 for i in range(0,len(data)) if data[i] <= x_q and data[i] >= oldx_q and Y[i] <= y_q and Y[i] >= oldy_q )
        
      elif oldx_q == 0:
        count = sum(1 for i in range(0,len(data)) if data[i] <= x_q and data[i] >= oldx_q and Y[i] <= y_q and Y[i] > oldy_q )
        
      elif oldy_q == 0:
        count = sum(1 for i in range(0,len(data)) if data[i] <= x_q and data[i] > oldx_q and Y[i] <= y_q and Y[i] >= oldy_q )
        
      else:
        count = sum(1 for i in range(0,len(data)) if data[i] <= x_q and data[i] > oldx_q and Y[i] <= y_q and Y[i] > oldy_q )
      if FY[i] != 0:
        FXY.append(count/FY[i])
      else:
        FXY.append(0)
      oldy_q = y_q
    oldx_q = x_q

  return(FXY)

def conditional_binning(data,Y,X_q,Y_q):
  FXY = []
  FY = []

  for y_q in Y_q:
    count = sum(1 for y in Y if y <= y_q )
    FY.append(count)

  for x_q, i, y_q in [(x_q, i, y_q) for x_q in X_q for i, y_q in enumerate(Y_q)]:
    count = sum(1 for x,y in zip(data, Y) if x <= x_q and y <= y_q)
        
    if FY[i] != 0:
      FXY.append(count/FY[i])
    else:
      FXY.append(0)
      

  return(FXY)
  
def binning(features_list, data, y):
    
  b = {"bin_edges":[]}

  y = y[0].to_list()
  Y_q = ranging(y,5)

  if len(features_list) > 1:
    combos = list(itertools.combinations(features_list,2))
  else:
    combos = []

  for combo in combos:
    bin_all = []

    data_feature = list(data[combo[0]])
    data_feature = scale(data_feature)
    data_feature = data_feature[0].to_list()
    
    X_q = ranging(data_feature,5)
    
    bin = conditional_binning(data_feature,y,X_q,Y_q)
    bin_all.extend(bin)

    data_feature = list(data[combo[1]])
    data_feature = scale(data_feature)
    data_feature = data_feature[0].to_list()
    
    X_q = ranging(data_feature,5)
    
    bin = conditional_binning(data_feature,y,X_q,Y_q)
    bin_all.extend(bin)

    b["bin_edges"].append(bin_all) 
    
  training_data = np.array(b["bin_edges"])
 
  return(training_data)
  
def create_testdata(X, Y):
  '''
  return test data to be fed into mlps
  '''
  test_data = []
  
  features_list = X.columns
  test = binning(features_list, X, Y)
  test_data.extend(test)
  
  return test_data

# Output recommendation results

### output all transformations per dataset

In [ ]:
# recommend all
def recommend(path, b, X, Y, qsa_):
  # iterate over all models
  models_ls = glob.glob(path)

  # res = []

  test_new = X.copy()
  # for one test data
  test_data = create_testdata(X, Y, b, qsa_)

  colnames = X.columns

  combos = list(itertools.combinations(colnames,2))

  row = []
  scores = []

  for m in models_ls:
    row.append(m.split('/')[-1][:-3])
    model = tf.python.keras.models.load_model(m)
    pred = model.predict(np.array(test_data), verbose=1)
    # the score for positive label
    scores.append(pred[:,1])

  result = pd.DataFrame(scores, columns = combos, index = row)
  
  # print(result.head())
  # print('Recommended transformation for ',test_files[i].split('/')[-1][:-4])
  # print('\n')

  dict_ = {}

  cnt = 0
  
  for combo in combos:
    sorted_result = result.sort_values(axis = 0, by = combo, ascending = False)
    rcm_trans = sorted_result[sorted_result[combo] >= 0.5].index.tolist()
    dict_[combo] = rcm_trans
    print(combo,': ',rcm_trans)
    if len(rcm_trans) > 0:
      cnt += 1

  if cnt == 0:
    print('No transformation is recommended for this dataset!!!')
    
  # res.append(dict_)
  
  # print('\n')

  return dict_, cnt

In [ ]:
# classifiers
def linear_regression(x_var, y_var):
  X_train, X_test, y_train, y_test = train_test_split(x_var, y_var, test_size=0.2, random_state=0)
  lr = LinearRegression()  
  lr.fit(X_train, y_train)
  y_pred = lr.predict(X_test)
  score = metrics.r2_score(y_test, y_pred)
  return(score)

def decision_tree(x_var, y_var):
  X_train, X_test, y_train, y_test = train_test_split(x_var, y_var, test_size=0.2, random_state=0)
  clf = tree.DecisionTreeRegressor(random_state = 0)
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  score = metrics.r2_score(y_test, y_pred)
  return(score)

## Replace features

In [ ]:
# binary transformation
def add(data, feature1, feature2):
  data_new = data.copy()
  data_new['new'] = data_new[feature1] + data_new[feature2]
  data_new.drop(axis=0,columns=[feature1,feature2],inplace=True)
  return(data_new)

def multiply(data, feature1, feature2):
  data_new = data.copy()
  data_new['new'] = data_new[feature1] * data_new[feature2]
  #data_new.drop(axis=0,columns=[feature1,feature2],inplace=True)
  return(data_new)

def subtract(data, feature1, feature2):
  data_new = data.copy()
  data_new['new'] = abs(data_new[feature1] - data_new[feature2])
  #data_new.drop(axis=0,columns=[feature1,feature2],inplace=True)
  return(data_new)


## Add features

In [ ]:
# transformations
def freq_add(data, feature):
  """convert the feature values to absolute value and then take the square root"""
  data_new = data.copy()
  given_feature = data_new[feature]
  frequency = collections.Counter(given_feature) 
  transformed_feature = [frequency[x] for x in given_feature] 
  feature_new = feature+'_new'
  data_new[feature_new] = transformed_feature
  return(data_new)

def sq_add(data, feature):
  """convert the feature values to absolute value and then take the square root"""
  data_new = data.copy()
  transformed_feature = data_new[feature]
  transformed_feature = transformed_feature**2
  feature_new = feature+'_new'
  data_new[feature_new] = transformed_feature
  return(data_new)

def sqr_add(data, feature):
  """convert the feature values to absolute value and then take the square root"""
  data_new = data.copy()
  transformed_feature = data_new[feature]
  transformed_feature = abs(transformed_feature)
  transformed_feature = np.sqrt(transformed_feature)
  feature_new = feature+'_new'
  data_new[feature_new] = transformed_feature
  return(data_new)

def log_add(data, feature):
  data_new = data.copy()
  transformed_feature = data_new[feature]
  transformed_feature = abs(transformed_feature)
  transformed_feature = np.where(transformed_feature == 0, 1e-9, transformed_feature)
  transformed_feature = np.log(transformed_feature)
  feature_new = feature+'_new'
  data_new[feature_new] = transformed_feature
  return(data_new)

def sigmoid_add(data, feature):
  data_new = data.copy()
  transformed_feature = data_new[feature]
  transformed_feature = 1/(1 + np.exp(-transformed_feature))
  feature_new = feature+'_new'
  data_new[feature_new] = transformed_feature
  return(data_new)

# Apply transformations

In [ ]:
def apply_trans(dict_, X, Y):

  score_before_ls = []
  score_after_ls = []
  diff_ls = []
  file_ls = []
  
  test_original = X.copy()
  label = Y

  test_original = scale_data(test_original)

  colnames = X.columns

  # apply transformation
  methods = {'add': add, 'subtract':subtract, 'multiply':multiply}
  test_new = test_original.copy()

  # dict_ = res[i]
  
  '''
  # apply all transformations on a feature
  for col in colnames:
    if len(dict_[col]) > 0:
      for i in dict_[col]:
        test_new = methods[i](test_new, col)
      test_new.drop(columns=[col],inplace = True)
  '''
  '''
  # apply one transformation on a feature
  for col in colnames:
    if len(dict_[col]) > 0:
      test_new = methods[dict_[col][0]](test_new, col)  
  

  '''
  # corresponding to one transformation per dataset
  combo = list(dict_.keys())[0]
  trans = dict_[combo]
  test_new = methods[trans](test_new, combo[0], combo[1])
  
  return test_new, label


In [ ]:
def eval(avg, file_name, test_original, test_new, label, *args):

  if avg == True:
    score_lnr0 = linear_regression(test_original, label)
    score_dtr0 = decision_tree(test_original, label)
    
    score_before = (score_lnr0 + score_dtr0) / 2

    score_lnr1 = linear_regression(test_new, label)
    score_dtr1 = decision_tree(test_new, label)
    
    score_after = (score_lnr1 + score_dtr1) / 2

  else:
    model = args[0]
    if model == 'lnr':
      score_before = linear_regression(test_original, label)
      score_after = linear_regression(test_new, label)

    elif model == 'dtr':
      score_before = decision_tree(test_original, label)
      score_after = decision_tree(test_new, label)
  
  if score_after > score_before:
    improve = ' improved'
  else:
    improve = ' not improved'

  diff = score_after - score_before

  print(file_name, ' original score: ',score_before,'; score after: ',score_after, '; difference: ', diff ,improve)
  # print(test_files[i].split('/')[-1][:-4], improve)

  return score_before, score_after, diff, file_name

In [ ]:
# combination of trasformations
def multiple_testing(path, b, n, model):
  score_before_ls = []
  score_after_ls = []
  diff_ls = []
  file_ls = []

  for i in range(len(X_ls)):
    X = X_ls[i].copy()
    Y = Y_ls[i].copy()
    #label = Y_label[i].copy()
    X_origin = scale_data(X)
    for j in range(n):
      #res, cnt = recommend(path, b, X, label)
      res, cnt = recommend(path, b, X, Y)
      if cnt > 0:
        X_new, Y = apply_trans(res, X, Y)
        X = X_new
    score_before, score_after, diff, file = eval(False, test_files[i].split('/')[-1][:-4], X_origin, X, Y, model)
    #if score_before <= 0.999 and score_before > 0:
    score_before_ls.append(score_before)
    score_after_ls.append(score_after)
    diff_ls.append(score_after - score_before)
    file_ls.append(test_files[i].split('/')[-1][:-4])

  return score_before_ls, score_after_ls, diff_ls, file_ls
